# Compare Bandstructures

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

In [ ]:
%matplotlib inline
from matplotlib.gridspec import GridSpec
from aiida.orm import load_node
import nanoribbon
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def make_plots(pk_list):
    # display progress bar
    progress = widgets.IntProgress(description='Plotting...', max=len(pk_list))
    display(progress)

    args_erang = [-3,3]

    fig = plt.figure()
    fig.set_size_inches(2.8*len(pk_list), 16)
    gs = GridSpec(5, len(pk_list))
    fig.text(0.05, 0.5, 'E(eV)', va='center', rotation='vertical')

    for i, pk in enumerate(pk_list):
        progress.value = i

        node = load_node(pk=int(pk))

        props = nanoribbon.get_all_properties(node)

        if i==0:
            ax1=fig.add_subplot(gs[1:5,i])
        else:
            ax1=fig.add_subplot(gs[1:5,i], sharey=ax_old)
            ax1.tick_params(axis='y', which='both',left='on',right='off', labelleft='off')

        allign=(props['homo']+props['lumo'])/2.0
        #print "plotting ({}/{})".format(i+1,n_bands)        
        nanoribbon.plot_b(props['hartree_calc_obj'], props['bands_calc_obj'], props['scf_calc_obj'], args_erang, allign, ax1)
    #    ax1.xaxis('off')
    #    ax1.tick_params(axis='x', which='both', bottom='off', top='off',  labelbottom='off')
        ax1.tick_params(axis='x', which='both', bottom='off', top='off' )
        labels = [item.get_text() for item in ax1.get_xticklabels()]
        labels[0]='$\Gamma$'
        labels[-1]='X'
        ax1.set_xticklabels(labels)

        ax1.set_xlabel('-Abs. magn.: {}$\mu_B$\n-Tot. magn.: {}$\mu_B$\n-Band gap: {:.3f} eV'.format(props['amag'],props['tmag'],props['gap']))

        ax2 = fig.add_subplot(gs[0:1,i])
        nanoribbon.plot_thumbnail(ax2, props['struct'].get_ase())

        ax_old = ax1

    progress.close()

In [ ]:
import urlparse
url = urlparse.urlsplit(jupyter_notebook_url)
params = urlparse.parse_qs(url.query)
if 'pk' in params.keys():
    make_plots(params['pk'])
else:
    print("Nothing to compare.")